In [408]:
import ast
from bs4 import BeautifulSoup
import csv
import json
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import re
import requests
from scipy import stats
import seaborn as sns
import sqlite3
from sqlite3 import Error
import time
import warnings
warnings.filterwarnings("ignore")

In [431]:
def product_url_finder(main_url, pageCount):
    product_url_list = []
    url = None
    for i in range(1, pageCount+1):
        number = str(i)
        url = main_url + "?pageSize=10&currentPage=" + number
        print("Scraping: " + url)
        r = requests.get(url)
        if r.status_code == 200:
            c = r.content
            soup = BeautifulSoup(c, 'html.parser')
            product_containers = soup.find_all("div", class_="css-1aj5qq4")
            for product_container in product_containers:
                product_loads = product_container.find_all("div", class_="css-dkxsdo")
            for product_load in product_loads:
                base_url = "https://www.sephora.com"
                product_links = product_load.find_all("a", class_="css-ix8km1")
                for product_link in product_links:
                    product_url = product_link["href"].replace(" ", "%20")
                    complete_url = base_url + product_url
                    product_url_list.append(complete_url)
        else:
            print(r.status_code)
    return product_url_list

In [410]:
product_list = product_url_finder("https://www.sephora.com/shop/moisturizing-cream-oils-mists", 84)

Scraping: https://www.sephora.com/shop/moisturizing-cream-oils-mists?pageSize=10&currentPage=1
Scraping: https://www.sephora.com/shop/moisturizing-cream-oils-mists?pageSize=10&currentPage=2
Scraping: https://www.sephora.com/shop/moisturizing-cream-oils-mists?pageSize=10&currentPage=3
Scraping: https://www.sephora.com/shop/moisturizing-cream-oils-mists?pageSize=10&currentPage=4
Scraping: https://www.sephora.com/shop/moisturizing-cream-oils-mists?pageSize=10&currentPage=5
Scraping: https://www.sephora.com/shop/moisturizing-cream-oils-mists?pageSize=10&currentPage=6
Scraping: https://www.sephora.com/shop/moisturizing-cream-oils-mists?pageSize=10&currentPage=7
Scraping: https://www.sephora.com/shop/moisturizing-cream-oils-mists?pageSize=10&currentPage=8
Scraping: https://www.sephora.com/shop/moisturizing-cream-oils-mists?pageSize=10&currentPage=9
Scraping: https://www.sephora.com/shop/moisturizing-cream-oils-mists?pageSize=10&currentPage=10
Scraping: https://www.sephora.com/shop/moisturizi

In [411]:
len(product_list)

837

In [412]:
product_series = pd.Series(product_list)

In [413]:
product_series.to_csv(r"/Users/florencialeoni/Documents/flatironDS/Mod2/i_feel_pretty/product_urls.csv")

In [423]:
def product_info_finder(url_list):
    products = {"brand": [], "name": [], "price": [], "n_of_reviews": [], 
               "n_of_loves": [], "review_score": [], "size": [], "clean_product": [], "category": []}

    for product_url in url_list:
        print("Product URL: " + product_url)
        r = requests.get(product_url)
        if r.status_code == 200:
            c = r.content
            soup = BeautifulSoup(c, 'html.parser')
            # Get a hold of the main product information container.
            main_container = soup.find("main", class_="css-1o80i28")
            
            # Get the product category.
            product_category = main_container.find("h1", class_="css-bnsadm")
            if product_category is None:
                products["category"].append("None")
            else:
                products["category"].append(product_category.text)
            
            # Get brand.
            product_brand = main_container.find("span", class_="css-euydo4").text
            products["brand"].append(product_brand)
            
            # Get product name.
            product_name = main_container.find("span", class_="css-0").text
            products["name"].append(product_name)
            
            # Get product price.
            product_price = main_container.find("div", class_="css-14hdny6").text
            products["price"].append(product_price.replace("$",""))
            
            # Get the product size. This is tricky, since it might be a single option,
            # multiple, or it might not have a size at all.
            
            product_size = main_container.find("div", class_="css-1qf1va5").span
            
            if product_size is not None:
                products["size"].append(product_size.text.replace("SIZE ", "").split("/")[0])
            else:
                product_size = main_container.find("button", class_="css-cl742e")
                if product_size is not None:
                    products["size"].append(product_size.text.split("/")[0])
                else:                
                    products["size"].append("None")
                
            # The the number of product loves.
            product_n_loves = main_container.find("div", class_="css-th1w6d").span.text
            products["n_of_loves"].append(product_n_loves.replace(" loves",""))
            
            # Find out if it's labeled "Clean at Sephora".
            product_clean = main_container.find_all(text="Clean at Sephora")
            if product_clean:
                products["clean_product"].append(1)
            else:
                products["clean_product"].append(0)

            # Get the number of reviews.
            product_n_reviews = main_container.find("span", class_="css-2rg6q7").text.replace(" reviews","").replace(" review", "").split("K")
            if len(product_n_reviews) > 1:
                product_n_reviews = int(product_n_reviews[0]) * 1000
            else:
                product_n_reviews = int(product_n_reviews[0])
                
            products["n_of_reviews"].append(product_n_reviews)
            
            # Get the score.
            product_score = 0.05*float(main_container.find("div", class_="css-1pjru6n")["style"].replace("width:", "").replace("%", ""))
            products["review_score"].append(product_score)
            
        else:
            print(r.status_code)
    return products

In [424]:
moisturizer_dict = product_info_finder(product_list)

Product URL: https://www.sephora.com/product/protini-tm-polypeptide-cream-P427421?icid2=products%20grid:p427421:product
Product URL: https://www.sephora.com/product/creme-de-la-mer-moisturizing-cream-P416341?icid2=products%20grid:p416341:product
Product URL: https://www.sephora.com/product/your-skin-but-better-cc-cream-spf-50-P411885?icid2=products%20grid:p411885:product
Product URL: https://www.sephora.com/product/the-water-cream-P418218?icid2=products%20grid:p418218:product
Product URL: https://www.sephora.com/product/facial-treatment-essence-P375849?icid2=products%20grid:p375849:product
Product URL: https://www.sephora.com/product/lala-retro-whipped-cream-P408645?icid2=products%20grid:p408645:product
Product URL: https://www.sephora.com/product/the-dewy-skin-cream-P441101?icid2=products%20grid:p441101:product
Product URL: https://www.sephora.com/product/virgin-marula-tm-luxury-facial-oil-P392245?icid2=products%20grid:p392245:product
Product URL: https://www.sephora.com/product/the-o

Product URL: https://www.sephora.com/product/glowstarter-mega-illuminating-moisturizer-P408739?icid2=products%20grid:p408739:product
Product URL: https://www.sephora.com/product/water-bank-moisture-cream-P420655?icid2=products%20grid:p420655:product
Product URL: https://www.sephora.com/product/tidal-brightening-enzyme-water-cream-P406938?icid2=products%20grid:p406938:product
Product URL: https://www.sephora.com/product/kale-spinach-hyaluronic-acid-age-prevention-cream-P411388?icid2=products%20grid:p411388:product
Product URL: https://www.sephora.com/product/watermelon-glow-sleeping-mask-P420160?icid2=products%20grid:p420160:product
Product URL: https://www.sephora.com/product/secret-sauce-clinically-advanced-miraculous-anti-aging-moisturizer-P421958?icid2=products%20grid:p421958:product
Product URL: https://www.sephora.com/product/goodnight-glow-retin-alt-sleeping-creme-P434546?icid2=products%20grid:p434546:product
Product URL: https://www.sephora.com/product/emerald-cannabis-sativa-he

Product URL: https://www.sephora.com/product/aqua-bomb-sleeping-mask-P433443?icid2=products%20grid:p433443:product
Product URL: https://www.sephora.com/product/dr-andrew-weil-for-origins-mega-bright-spf-30-skin-tone-correcting-oil-free-moisturizer-P385253?icid2=products%20grid:p385253:product
Product URL: https://www.sephora.com/product/the-renewal-oil-P417241?icid2=products%20grid:p417241:product
Product URL: https://www.sephora.com/product/peat-miracle-revital-cream-P412440?icid2=products%20grid:p412440:product
Product URL: https://www.sephora.com/product/the-moisturizing-cool-gel-cream-P429637?icid2=products%20grid:p429637:product
Product URL: https://www.sephora.com/product/facial-spray-with-aloe-herbs-rosewater-P440482?icid2=products%20grid:p440482:product
Product URL: https://www.sephora.com/product/hydro-dynamic-ultimate-moisture-P268700?icid2=products%20grid:p268700:product
Product URL: https://www.sephora.com/product/ginzing-spf-40-energy-boosting-tinted-moisturizer-P420966?ic

Product URL: https://www.sephora.com/product/black-label-detox-bb-beauty-balm-P374568?icid2=products%20grid:p374568:product
Product URL: https://www.sephora.com/product/phoenix-cell-regenerating-facial-oil-P400204?icid2=products%20grid:p400204:product
Product URL: https://www.sephora.com/product/all-day-hydrator-hydrate-glow-P438617?icid2=products%20grid:p438617:product
Product URL: https://www.sephora.com/product/photo-finish-primerizer-P421232?icid2=products%20grid:p421232:product
Product URL: https://www.sephora.com/product/vitamin-enriched-face-base-priming-moisturizer-mini-P441804?icid2=products%20grid:p441804:product
Product URL: https://www.sephora.com/product/colored-clay-cc-undereye-corrector-P387175?icid2=products%20grid:p387175:product
Product URL: https://www.sephora.com/product/hyaluronic-marine-oil-free-moisture-cushion-P416755?icid2=products%20grid:p416755:product
Product URL: https://www.sephora.com/product/photo-finish-primer-oil-P404235?icid2=products%20grid:p404235:p

Product URL: https://www.sephora.com/product/volcanic-elixir-P430714?icid2=products%20grid:p430714:product
Product URL: https://www.sephora.com/product/100-squalane-oil-P416564?icid2=products%20grid:p416564:product
Product URL: https://www.sephora.com/product/bb-cushion-pore-blur-P420658?icid2=products%20grid:p420658:product
Product URL: https://www.sephora.com/product/5-in-1-face-cream-spf-30-P263006?icid2=products%20grid:p263006:product
Product URL: https://www.sephora.com/product/superdefense-spf-20-age-defense-moisturizer-very-dry-to-dry-combination-P223707?icid2=products%20grid:p223707:product
Product URL: https://www.sephora.com/product/benefiance-wrinkleresist24-day-emulsion-broad-spectrum-spf-18-P173614?icid2=products%20grid:p173614:product
Product URL: https://www.sephora.com/product/dramatically-different-hydrating-jelly-P432241?icid2=products%20grid:p432241:product
Product URL: https://www.sephora.com/product/100-pure-argan-oil-light-P380805?icid2=products%20grid:p380805:pro

...

Product URL: https://www.sephora.com/product/hungarian-thermal-water-mineral-rich-moisturizer-P427637?icid2=products%20grid:p427637:product
Product URL: https://www.sephora.com/product/balancing-moisturizer-broad-spectrum-spf-15-pa-P379902?icid2=products%20grid:p379902:product
Product URL: https://www.sephora.com/product/future-solution-lx-total-regenerating-cream-P423169?icid2=products%20grid:p423169:product
Product URL: https://www.sephora.com/product/vital-treatment-essence-for-hydrating-P408268?icid2=products%20grid:p408268:product
Product URL: https://www.sephora.com/product/hydrating-moisturizer-for-dry-skin-P423740?icid2=products%20grid:p423740:product
Product URL: https://www.sephora.com/product/repairwear-sculpting-night-cream-P392241?icid2=products%20grid:p392241:product
Product URL: https://www.sephora.com/product/pureness-moisturizing-gel-cream-P173645?icid2=products%20grid:p173645:product
Product URL: https://www.sephora.com/product/rejuvenating-lift-for-neck-decollete-P29

Product URL: https://www.sephora.com/product/c-collagen-perfect-skin-set-refresh-mist-P411361?icid2=products%20grid:p411361:product
Product URL: https://www.sephora.com/product/the-radiance-brightening-vitamin-e-rosehip-seed-facial-oil-P438629?icid2=products%20grid:p438629:product
Product URL: https://www.sephora.com/product/dry-erase-ultra-calming-face-cream-P397665?icid2=products%20grid:p397665:product
Product URL: https://www.sephora.com/product/make-a-difference-trade-plus-43-rejuvenating-treatment-P376633?icid2=products%20grid:p376633:product
Product URL: https://www.sephora.com/product/no-4-vanilla-silk-hydrating-lotion-P428643?icid2=products%20grid:p428643:product
Product URL: https://www.sephora.com/product/complexion-perfecting-hydrator-P440063?icid2=products%20grid:p440063:product
Product URL: https://www.sephora.com/product/oil-control-mattifying-mist-P421949?icid2=products%20grid:p421949:product
Product URL: https://www.sephora.com/product/and-coco-bliss-intensive-repair-P4

Product URL: https://www.sephora.com/product/cactus-flower-tibetan-ginseng-hydrating-mist-P426825?icid2=products%20grid:p426825:product
Product URL: https://www.sephora.com/product/multi-miracle-glow-P433524?icid2=products%20grid:p433524:product
Product URL: https://www.sephora.com/product/3-in-1-cleansing-oil-P416596?icid2=products%20grid:p416596:product
Product URL: https://www.sephora.com/product/white-lucent-day-emulsion-broad-spectrum-spf-23-P443377?icid2=products%20grid:p443377:product
Product URL: https://www.sephora.com/product/advanced-time-zone-age-reversing-line-wrinkle-hydrating-gel-oil-free-P380006?icid2=products%20grid:p380006:product
Product URL: https://www.sephora.com/product/yuza-sorbet-night-reinforced-nighttime-emulsion-P406940?icid2=products%20grid:p406940:product
Product URL: https://www.sephora.com/product/pure-nurture-facial-water-P408233?icid2=products%20grid:p408233:product
Product URL: https://www.sephora.com/product/glow-creme-illuminating-face-cream-P417244

Product URL: https://www.sephora.com/product/cosmic-maracuja-concentrated-face-balm-P433154?icid2=products%20grid:p433154:product
Product URL: https://www.sephora.com/product/calming-lavender-mist-P423731?icid2=products%20grid:p423731:product
Product URL: https://www.sephora.com/product/oil-control-daily-moisturizer-P440288?icid2=products%20grid:p440288:product
Product URL: https://www.sephora.com/product/even-better-skin-tone-correcting-lotion-broad-spectrum-spf-20-P309226?icid2=products%20grid:p309226:product
Product URL: https://www.sephora.com/product/blu-mediterraneo-italian-resort-moisturizing-face-lotion-P408979?icid2=products%20grid:p408979:product
Product URL: https://www.sephora.com/product/aqua-reotier-ultra-thirst-quenching-cream-moisturizer-P429938?icid2=products%20grid:p429938:product
Product URL: https://www.sephora.com/product/nuddy-ultra-hydrating-repair-cr-me-P430834?icid2=products%20grid:p430834:product
Product URL: https://www.sephora.com/product/time-retreat-radian

Product URL: https://www.sephora.com/product/multi-biotic-moisturizer-P443832?icid2=products%20grid:p443832:product
Product URL: https://www.sephora.com/product/skin-soothing-hydrating-lotion-P443368?icid2=products%20grid:p443368:product
Product URL: https://www.sephora.com/product/squalane-oil-P443839?icid2=products%20grid:p443839:product
Product URL: https://www.sephora.com/product/turmeric-moisturizer-P443835?icid2=products%20grid:p443835:product


In [425]:
moisturizers_df = pd.DataFrame.from_dict(moisturizer_dict)

In [426]:
moisturizers_df.to_csv(r"/Users/florencialeoni/Documents/flatironDS/Mod2/i_feel_pretty/moisturizers.csv")

In [429]:
skincare_df = pd.read_excel("/Users/florencialeoni/Documents/flatironDS/Mod2/i_feel_pretty/moisturizers.xls")

In [432]:
treatments_url = product_url_finder("https://www.sephora.com/shop/facial-treatments", 58)

Scraping: https://www.sephora.com/shop/facial-treatments?pageSize=10&currentPage=1
Scraping: https://www.sephora.com/shop/facial-treatments?pageSize=10&currentPage=2
Scraping: https://www.sephora.com/shop/facial-treatments?pageSize=10&currentPage=3
Scraping: https://www.sephora.com/shop/facial-treatments?pageSize=10&currentPage=4
Scraping: https://www.sephora.com/shop/facial-treatments?pageSize=10&currentPage=5
Scraping: https://www.sephora.com/shop/facial-treatments?pageSize=10&currentPage=6
Scraping: https://www.sephora.com/shop/facial-treatments?pageSize=10&currentPage=7
Scraping: https://www.sephora.com/shop/facial-treatments?pageSize=10&currentPage=8
Scraping: https://www.sephora.com/shop/facial-treatments?pageSize=10&currentPage=9
Scraping: https://www.sephora.com/shop/facial-treatments?pageSize=10&currentPage=10
Scraping: https://www.sephora.com/shop/facial-treatments?pageSize=10&currentPage=11
Scraping: https://www.sephora.com/shop/facial-treatments?pageSize=10&currentPage=12
S

In [434]:
treatment_series = pd.Series(treatments_url)

In [435]:
treatment_series.to_csv(r"/Users/florencialeoni/Documents/flatironDS/Mod2/i_feel_pretty/treatment_urls.csv")